In [ ]:
pd.set_option('display.max_colwidth',-1)

In [ ]:
# Importing or Exporting from excel can take only a max of 32767 characters per cell. Though Excel sheet from Craig contains max of 32767 chars, when imported into excel spaces and line breaks are created as characters and hence entire data is not imported and it is truncated
# Issue can be solved by directly importing from database etc

In [ ]:
# https://github.com/akoumjian/datefinder
import sys 
!{sys.executable} -m pip install datefinder 

In [ ]:
import pandas as pd
import re
import datetime
from datetime import *

In [ ]:
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None 

In [ ]:
import pickle
ClinicalNotes = pd.read_pickle("Py DataFrames/ClinicalNotes.pkl")

In [ ]:
ClinicalNotes = pd.read_excel('Craig_2019Aug7th.xlsx')

# Strip white space
# ClinicalNotes["Text"] = ClinicalNotes["Text"].str.strip()

# convert text to lowercase
ClinicalNotes["Text"] = ClinicalNotes["Text"].str.lower()

# Convert index to column
ClinicalNotes['Row'] = ClinicalNotes.index

# Remove records with source type = ADT, Lab & Order
ClinicalNotes = ClinicalNotes[ClinicalNotes.SourceType.isin(["User Entered","Documentation"])]

# Remove records based on number of words and if there is a number present or not and based on length of sentence
# For eg: Bmi>40 is a single word has a number but length is less
# This example has number of words=1 but length of sentence is long "wprnbwygkmfydj3ijapkf7y15yopqc0lgey2a0iryfkxbvs1zeyjm5c\r\n0vt1iwgfwijef62vbd2j+9ofnokh+moppxxdv7tzunjjcr6/peo0fticiqsb4fli7wudgfonhzab\r\nixs3biz"

# Number of words in a sentence
# df['totalwords'] = df['col'].str.split().str.len() #if /r or /n is present it is considered as space 
# df['totalwords'] = df['col'].str.count(' ') + 1 #if space is present or not
# df['totalwords'] = [len(x.split()) for x in df['col'].tolist()] #
ClinicalNotes['TotalWords'] = ClinicalNotes['Text'].str.count(' ') + 1

# Find the length of the sentence
ClinicalNotes['TextLength'] = ClinicalNotes['Text'].str.len() 

# Check if the Text has a Numeric in it or not
ClinicalNotes['NumericInText'] = ClinicalNotes['Text'].str.contains(("\d"), regex=True)

# Sort data
ClinicalNotes = ClinicalNotes.sort_values(by=['TotalWords','TextLength','NumericInText'], ascending=[True, True, True])

# Remove rows based on the conditions
Removed_ClinicalNotes = ClinicalNotes[(((ClinicalNotes.TotalWords<=5) & (ClinicalNotes.TextLength>500)) | (ClinicalNotes.NumericInText==False))]

## Remove 
# TotalWords<=3 & TextLength>300
# NumericInText is not present
ClinicalNotes = ClinicalNotes[~(((ClinicalNotes.TotalWords<=3) & (ClinicalNotes.TextLength>500)) | (ClinicalNotes.NumericInText==False))]

# Export data
ClinicalNotes.to_excel('Py DataFrames/ClinicalNotes.xlsx',index=False)
ClinicalNotes.to_pickle("Py DataFrames/ClinicalNotes.pkl")

In [ ]:
ClinicalNotes.describe()

In [ ]:
# Identifies only single date present
# Check_head['Date'] = Check_head['Text'].str.extract(r'(\d+/\d+)', expand=True)

# Identifies only if 2 dates are present
# Check_head['Date'] = Check_head.Text.str.extract(r'(\d+/\d+)(\D+)(\d+/\d+)', expand=True)

In [ ]:
# # Find dates in m/d/y format
# Check_head['Month_Date_Year'] = Check_head["Text"].str.findall(r'\d+/\d+/\d+')

# # Find dates in m/d format
# Check_head['Month_date'] = Check_head["Text"].str.findall(r'\d+/\d+')

In [ ]:
# Row - 28025
#  wbc 
# 13.6 k/mcl (high) 
# 08/06/2019 16:22 cdt 
    ---So data needs to be exploded only after dates conversion 
# if Text is in such a way that "Any text "\n" Value" then replace \n with space --- This needs to be done after removing dates

In [ ]:
# 	systolic blood pressure	101 mm
# systolic blood pressure\t101 mmhg\t\n\t

# '\t' needs to be replaced with " " space

In [ ]:
# Split datasets and once corrected attach back - 
# One efficient way might be to search for the string and extract only those that matches that string  

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes.Text.str.split('\r\n\r\n').tolist(), index=ClinicalNotes.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\r\n').tolist(), index=ClinicalNotes_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\n').tolist(), index=ClinicalNotes_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\r').tolist(), index=ClinicalNotes_Explode.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']

### Explode the Text column into multiple rows
# We start with creating a new dataframe from the series with EmployeeId as the index
ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes.Text.str.split('[#!?.]\s').tolist(), index=ClinicalNotes.Row).stack()
# We now want to get rid of the secondary index
ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# The final step is to set the column names as we want them
ClinicalNotes_Explode.columns = ['Row', 'Text']


# donot split that data based on full stops as ClinicalNote_extract_candidates_numeric function will take care of it and 
# if start or end of line is date we will miss it
# ### Explode the Text column into multiple rows - Temp = 101.2 will be split at decimals so space to be added after the 
# # We start with creating a new dataframe from the series with EmployeeId as the index
# ClinicalNotes_Explode = pd.DataFrame(ClinicalNotes_Explode.Text.str.split('\. ').tolist(), index=ClinicalNotes_Explode.Row).stack()
# # We now want to get rid of the secondary index
# ClinicalNotes_Explode = ClinicalNotes_Explode.reset_index([0, 'Row'])
# # The final step is to set the column names as we want them
# ClinicalNotes_Explode.columns = ['Row', 'Text']

In [ ]:
ClinicalNotes_Explode.to_pickle("Py DataFrames/ClinicalNotes_Explode.pkl")

In [ ]:
ClinicalNotes_Explode

In [ ]:
# 4388, 38839
# ClinicalNotes['Text'].iloc[38839]
ClinicalNotes[ClinicalNotes['Row']==38839].Text

In [ ]:
# Extract only those strings with wbc
wbc_Explode = ClinicalNotes_Explode[ClinicalNotes_Explode["Text"].str.contains("wbc")]

In [ ]:
# Shuffle rows
wbc_Explode = wbc_Explode.sample(frac=1)

In [ ]:
# Remove rows that have 

In [ ]:
wbc_Explode.to_csv('wbc_Explode.csv', index=False)
wbc_Explode.to_pickle("wbc_Explode.pkl")

In [ ]:
wbc_Explode

In [ ]:
Result = wbc_Explode[~(wbc_Explode["Text"].str.contains(("wbc \d"), regex=True))]
Result = Result[~(Result["Text"].str.contains(("wbc: \d"), regex=True))]

In [ ]:
Result

In [ ]:
Result.shape

In [ ]:
textToSearch = "date of service from sep08 to 8/6/19 to 9/8/19 to 08/05/19 17:26:00 CDT and not 6/7"

In [ ]:
import re


In [ ]:
import datefinder
matches = datefinder.find_dates(textToSearch)
for match in matches:
    print(match)

In [ ]:
def ChangeDateTimeFormat(TextwithDates):
    for match in re.findall(r'/\d{4}', TextwithDates):
        match = re.sub('/','' , match)
        #convert match to new format
        datetime_object = datetime.strptime(match, "%Y")
        dateNewFormat = datetime_object.strftime("%y")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

    # convert %m/%d/%y formatted texts to %m/%d
    for match in re.findall(r'\d+/\d+/\d+', TextwithDates):
        #convert match to new format
        datetime_object = datetime.strptime(match, "%m/%d/%y")
        dateNewFormat = datetime_object.strftime("%m/%d")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
    
    # convert %m/%d formatted texts to %B%d
    for match in re.findall(r'\d+/\d+ \d+:\d+:\d+', TextwithDates):
        datetime_object = datetime.strptime(match, "%m/%d %H:%M:%S")
        dateNewFormat = datetime_object.strftime("%B%d_%H:%M:%S")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
    
    for match in re.findall(r'\d+/\d+ \d+:\d+', TextwithDates):
        datetime_object = datetime.strptime(match, "%m/%d %H:%M")
        dateNewFormat = datetime_object.strftime("%B%d_%H:%M")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
    
    for match in re.findall(r'\d+/\d+', TextwithDates):
        datetime_object = datetime.strptime(match, "%m/%d")
        dateNewFormat = datetime_object.strftime("%B%d")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
    
    MonDD = r"jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
    # First convert jan to january etc
    for match in re.findall(MonDD, TextwithDates):
        datetime_object = datetime.strptime(match,"%b%d")
        dateNewFormat = datetime_object.strftime("%B%d ")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
    Mon_DD = r"jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sept \d+|oct \d+|nov \d+|dec \d+"
    for match in re.findall(Mon_DD, TextwithDates):
        datetime_object = datetime.strptime(match,"%b %d")
        dateNewFormat = datetime_object.strftime("%B%d ")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

    MonthDD = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+"
    for match in re.findall(MonthDD, TextwithDates):
        datetime_object = datetime.strptime(match,"%B%d")
        dateNewFormat = datetime_object.strftime("%B%d ")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        
    Month_DD = r"january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
    for match in re.findall(Month_DD, TextwithDates):
        datetime_object = datetime.strptime(match,"%B %d")
        dateNewFormat = datetime_object.strftime("%B%d ")
        #substitute the old date with the new
        TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
    return(TextwithDates)

In [ ]:
# function to pass if there is an error - When there is an error the whole row is passed, 728 errors are present
def ErrorINChangeDateTimeFormat(TextwithDates):
    try:
        for match in re.findall(r'/\d{4}', TextwithDates):
            match = re.sub('/','' , match)
            #convert match to new format
            datetime_object = datetime.strptime(match, "%Y")
            dateNewFormat = datetime_object.strftime("%y")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d/%y formatted texts to %m/%d
        for match in re.findall(r'\d+/\d+/\d+', TextwithDates):
            #convert match to new format
            datetime_object = datetime.strptime(match, "%m/%d/%y")
            dateNewFormat = datetime_object.strftime("%m/%d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d formatted texts to %B%d
        for match in re.findall(r'\d+/\d+ \d+:\d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%d_%H:%M:%S")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        for match in re.findall(r'\d+/\d+ \d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%d_%H:%M")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        for match in re.findall(r'\d+/\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d")
            dateNewFormat = datetime_object.strftime("%B%d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        MonDD = r"jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
        # First convert jan to january etc
        for match in re.findall(MonDD, TextwithDates):
            datetime_object = datetime.strptime(match,"%b%d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        Mon_DD = r"jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sept \d+|oct \d+|nov \d+|dec \d+"
        for match in re.findall(Mon_DD, TextwithDates):
            datetime_object = datetime.strptime(match,"%b %d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        MonthDD = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+"
        for match in re.findall(MonthDD, TextwithDates):
            datetime_object = datetime.strptime(match,"%B%d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        Month_DD = r"january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
        for match in re.findall(Month_DD, TextwithDates):
            datetime_object = datetime.strptime(match,"%B %d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        return(TextwithDates)
    except Exception:
        pass

In [ ]:
# If error then return the original text itself
def ChangeDateTimeFormat(TextwithDates):
    try:
        for match in re.findall(r'/\d{4}', TextwithDates):
            match = re.sub('/','' , match)
            #convert match to new format
            datetime_object = datetime.strptime(match, "%Y")
            dateNewFormat = datetime_object.strftime("%y")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d/%y formatted texts to %m/%d
        for match in re.findall(r'\d+/\d+/\d+', TextwithDates):
            #convert match to new format
            datetime_object = datetime.strptime(match, "%m/%d/%y")
            dateNewFormat = datetime_object.strftime("%m/%d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        # convert %m/%d formatted texts to %B%d
        for match in re.findall(r'\d+/\d+ \d+:\d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M:%S")
            dateNewFormat = datetime_object.strftime("%B%d_%H:%M:%S")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        for match in re.findall(r'\d+/\d+ \d+:\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d %H:%M")
            dateNewFormat = datetime_object.strftime("%B%d_%H:%M")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        for match in re.findall(r'\d+/\d+', TextwithDates):
            datetime_object = datetime.strptime(match, "%m/%d")
            dateNewFormat = datetime_object.strftime("%B%d")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        MonDD = r"jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
        # First convert jan to january etc
        for match in re.findall(MonDD, TextwithDates):
            datetime_object = datetime.strptime(match,"%b%d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        Mon_DD = r"jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sept \d+|oct \d+|nov \d+|dec \d+"
        for match in re.findall(Mon_DD, TextwithDates):
            datetime_object = datetime.strptime(match,"%b %d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        MonthDD = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+"
        for match in re.findall(MonthDD, TextwithDates):
            datetime_object = datetime.strptime(match,"%B%d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)

        Month_DD = r"january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
        for match in re.findall(Month_DD, TextwithDates):
            datetime_object = datetime.strptime(match,"%B %d")
            dateNewFormat = datetime_object.strftime("%B%d ")
            #substitute the old date with the new
            TextwithDates = re.sub(match, dateNewFormat, TextwithDates)
        return(TextwithDates)
    except Exception:
        return(TextwithDates)

In [ ]:
textToSearch = "08/05/19 17:26:00 CDT 08/05 4:05 9/8/19 and 12/8/2019 temp 101 january8th and september 6 2019 march 8th xxx jan10th xyz jan 8th  na 140 (aug 07) 143 (aug 06) 141 (aug 05) " # 

# MONTHS_PATTERN = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+|january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+|jan\d+|jan \d+|feb\d+|feb \d+|mar\d+|mar \d+|apr\d+|apr \d+|may\d+|may \d+|jun\d+|jun \d+|jul\d+|jul \d+|aug\d+|aug \d+|sep\d+|sept \d+|oct\d+|oct \d+|nov\d+|nov \d+|dec\d+|dec \d+"
# # MONTHS_PATTERN = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+"
# # MONTHS_PATTERN = r"january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"

# MONTHS = r"jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec"
# MONTHS = r"jan\d+|jan \d+|feb\d+|feb \d+|mar\d+|mar \d+|apr\d+|apr \d+|may\d+|may \d+|jun\d+|jun \d+|jul\d+|jul \d+|aug\d+|aug \d+|sep\d+|sept \d+|oct\d+|oct \d+|nov\d+|nov \d+|dec\d+|dec \d+"
MonDD = r"jan\d+|feb\d+|mar\d+|apr\d+|may\d+|jun\d+|jul\d+|aug\d+|sep\d+|oct\d+|nov\d+|dec\d+"
# First convert jan to january etc
for match in re.findall(MonDD, textToSearch):
    datetime_object = datetime.strptime(match,"%b%d")
    dateNewFormat = datetime_object.strftime("%B%d ")
    #substitute the old date with the new
    textToSearch = re.sub(match, dateNewFormat, textToSearch)

Mon_DD = r"jan \d+|feb \d+|mar \d+|apr \d+|may \d+|jun \d+|jul \d+|aug \d+|sept \d+|oct \d+|nov \d+|dec \d+"
for match in re.findall(Mon_DD, textToSearch):
    datetime_object = datetime.strptime(match,"%b %d")
    dateNewFormat = datetime_object.strftime("%B%d ")
    #substitute the old date with the new
    textToSearch = re.sub(match, dateNewFormat, textToSearch)

MonthDD = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+"
for match in re.findall(MonthDD, textToSearch):
    datetime_object = datetime.strptime(match,"%B%d")
    dateNewFormat = datetime_object.strftime("%B%d ")
    #substitute the old date with the new
    textToSearch = re.sub(match, dateNewFormat, textToSearch)

Month_DD = r"january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+"
for match in re.findall(Month_DD, textToSearch):
    datetime_object = datetime.strptime(match,"%B %d")
    dateNewFormat = datetime_object.strftime("%B%d ")
    #substitute the old date with the new
    textToSearch = re.sub(match, dateNewFormat, textToSearch)
print(textToSearch)

In [ ]:
for match in re.findall(MONTHS_PATTERN, textToSearch):
    datetime_object = datetime.strptime(match, "%B%d")
    dateNewFormat = datetime_object.strftime("%B%d ")
    #substitute the old date with the new
    textToSearch = re.sub(match, dateNewFormat, textToSearch)
print(textToSearch)

In [ ]:
ClinicalNotes.apply(lambda x: ChangeTime(x.Text), axis=1)

In [ ]:
# 24889 - [july 15, august 2004, july 2009, november 2007, june 2012, january 2018] 
# 21572    [july 18, august 3, august 6, august 4, august 3, august 3]
# 30687    [july 21, february 2019]  
MONTHS_PATTERN = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+|january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+|jan\d+|jan \d+|feb\d+|feb \d+|mar\d+|mar \d+|apr\d+|apr \d+|may\d+|may \d+|jun\d+|jun \d+|jul\d+|jul \d+|aug\d+|aug \d+|sep\d+|sept \d+|oct\d+|oct \d+|nov\d+|nov \d+|dec\d+|dec \d+"

ClinicalNotes["Text"].str.findall(MONTHS_PATTERN)

In [ ]:
# remove rows from ClinicalNotes
ClinicalNotes_chk = ClinicalNotes[~(ClinicalNotes.Row.isin([7268,24917,776,7956,6236,7476,36225,38732,38832,3097,4411,4723,7674,9654,4722,7743,9688,9675,9681,22572,33745,23004,7746,5431,9022,9030,8602,24427,9071,9072,5604,23914,7406,9070,22172,2534,3897,7441,32772,1943,8684,7452,1635,36088,5635]))]

In [ ]:
ClinicalNotes.head(5)

In [ ]:
chk = ClinicalNotes[(ClinicalNotes.Row.isin([7268,24917,776,7956,6236,7476,36225,38732,38832,3097,4411,4723,7674,9654,4722,7743,9688,9675,9681,22572,33745,23004,7746,5431,9022,9030,8602,24427,9071,9072,5604,23914,7406,9070,22172,2534,3897,7441,32772,1943,8684,7452,1635,36088,5635]))]

In [ ]:
# Check for errors where the 
ClinicalNotes_error = ClinicalNotes
ClinicalNotes_error.Text = ClinicalNotes_error.apply(lambda x: ChangeDateFormat(x.Text), axis=1)
ClinicalNotes_error = ClinicalNotes_error[pd.isnull(ClinicalNotes_error.Text)]
Rows_error = ClinicalNotes_error.Row.tolist()

In [ ]:
ClinicalNotes_error = ClinicalNotes[(ClinicalNotes.Row.isin(Rows_error))]

In [ ]:
ClinicalNotes_error['TextFunction'] = ClinicalNotes_error.apply(lambda x: ChangeDateFormat(x.Text), axis=1)

In [ ]:
ClinicalNotes_error.to_excel('Py DataFrames/ClinicalNotes_error.xlsx',index=False)

In [ ]:
textToSearch = "date of service from 8/6 to 9/8/19 "
textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019"
textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019"
textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019 in 101.4 as 1400" # year 
textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019 in 101.4 as @1400"
textToSearch = "date of service from 8/6 to 9/8/19 and 12/8/2019 and september 6th 2019 in 101.4 as 6000"
textToSearch = "08/05/19 17:26:00 CDT 08/05 4:05 9/8/19 and 12/8/2019 temp 101 and september 6 2019 march 8th xxx jan10th xyz jan 8th  na 140 (aug 07) 143 (aug 06) 141 (aug 05) " # 
textToSearch = "scr remains stable 1.6-1.9. ply holding at 180s, ast/alt 76/51" # 
textToSearch = "16/6 per prog note: her work up for infectious colitis thus far has been negative" #If date is in dd/mm format the code ignores or doesn't convert 
textToSearch = "9/26-\nmica 9/27-\nceftriaxone 9/24-9/26\nmero 9/229/24\n\n9/27 id on board" #In this case, 9/229/24 it is supposed to be 9/22 and 9/24 but since there is no gap etc between the two it is considered as an error
textToSearch = "percocet 5/325\n\npmh: cad, chf, gerd, htn" 
# In row 4722 - looks like only of its date is written in dd/mm format
# In row 22572 - check again
# In row 33745 - 169382/1283521

In [ ]:
# 5741, 7268, 24917, 776,28281
# 9615,33876, 3099
textToSearch = ClinicalNotes[ClinicalNotes.Row==3099].Text
textToSearch

In [ ]:
textToSearch = ClinicalNotes[ClinicalNotes.Row==33876].Text

In [ ]:
textToSearch = "08/05/19 17:26:00 CDT 08/05 4:05 9/8/19 and 12/8/2019 temp 101 and september 6 2019 march 8th xxx jan10th xyz jan 8th  na 140 (aug 07) 143 (aug 06) 141 (aug 05) " # 

In [ ]:
MONTHS_PATTERN = r"january\d+|february\d+|march\d+|april\d+|may\d+|june\d+|july\d+|august\d+|september\d+|october\d+|november\d+|december\d+|january \d+|february \d+|march \d+|april \d+|may \d+|june \d+|july \d+|august \d+|september \d+|october \d+|november \d+|december \d+|jan\d+|jan \d+|feb\d+|feb \d+|mar\d+|mar \d+|apr\d+|apr \d+|may\d+|may \d+|jun\d+|jun \d+|jul\d+|jul \d+|aug\d+|aug \d+|sep\d+|sept \d+|oct\d+|oct \d+|nov\d+|nov \d+|dec\d+|dec \d+"
re.findall(MONTHS_PATTERN, textToSearch)

In [ ]:
import datefinder
matches = datefinder.find_dates(textToSearch)
for match in matches:
    print(match)

In [ ]:
ChangeDateFormat(textToSearch)

In [ ]:
ChangeDateFormat(textToSearch)

In [ ]:
ChangeYYYYToYY(textToSearch)

In [ ]:
ChangeMMDDYYtoMMDD(textToSearch)

In [ ]:
ChangeMMDDtoMonthDD(textToSearch)

In [ ]:
ChangeTime(textToSearch)

In [ ]:
ClinicalNotes.apply(lambda x: ChangeDateTimeFormat(x.Text), axis=1)

In [ ]:
ClinicalNotes_error.apply(lambda x: ChangeDateFormat(x.Text), axis=1)

In [ ]:
# error at index 7268